In [47]:
import geopandas as gpd
import rioxarray as rxr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import pandas as pd
import seaborn as sns

from src import constants
from src.utils import raster_utils

# 0. Read in and overlay disturbance + burn data

In [26]:
degrade_pts = gpd.read_feather(constants.RESULTS_PATH / "radd_treatment_singlelayer.feather")
rst = rxr.open_rasterio(constants.DATA_PATH / "annual_burndate" / "annual_burndate.tif")

In [27]:
xs = degrade_pts.t2_geom.x.values
ys = degrade_pts.t2_geom.y.values

rst_x_idxs = raster_utils.get_idx(rst.x.data, xs)
rst_y_idxs = raster_utils.get_idx(rst.y.data, ys)

In [28]:
yrs = [19, 20, 21, 22]

for i, y in enumerate(yrs):
    dt = rst.data[i, rst_y_idxs, rst_x_idxs]
    degrade_pts[f"burn_day_{y}"] = dt
    degrade_pts[f"burn_bool_{y}"] = np.logical_and(
        dt != 0, np.logical_and(
        dt + (y * 1000) > degrade_pts["t1_yydoy"],
        dt + (y * 1000) < degrade_pts["t2_yydoy"],
        ))

degrade_pts["interceding_burn"] = degrade_pts[[f"burn_bool_{y}" for y in yrs]].any(axis=1)

### 0.1 Bucket PAI according to vertical thirds of pre-disturbance height

In [29]:
print(len(degrade_pts[degrade_pts["interceding_burn"]]))
with pd.option_context("mode.chained_assignment", None):
    degrade_pts["t1_mid_tch_bucket"] = degrade_pts.t1_rh_98_a0.apply(lambda x: int((x / 3) // 5 * 5))
    degrade_pts["t1_upper_pai"] = np.nan
    degrade_pts["t2_upper_pai"] = np.nan
    degrade_pts["t1_lower_pai"] = np.nan
    degrade_pts["t2_lower_pai"] = np.nan
    for el in degrade_pts.t1_mid_tch_bucket.unique():
        idx = degrade_pts.t1_mid_tch_bucket == el
        degrade_pts["t1_upper_pai"][idx] = degrade_pts[f"t1_pai_{el}"][idx] - degrade_pts.t1_pai_100[idx]
        degrade_pts["t2_upper_pai"][idx] = degrade_pts[f"t2_pai_{el}"][idx] - degrade_pts.t2_pai_100[idx]

        degrade_pts["t1_lower_pai"][idx] = degrade_pts.t1_pai_0 - degrade_pts[f"t1_pai_{el}"][idx] 
        degrade_pts["t2_lower_pai"][idx] = degrade_pts.t2_pai_0 - degrade_pts[f"t2_pai_{el}"][idx] 

836


# 1. Plotting

In [ ]:
import matplotlib
red = matplotlib.cm.get_cmap("coolwarm")(1.0)
blue = matplotlib.cm.get_cmap("coolwarm")(0.0)
red1 = "tab:red"
blue1 = "tab:blue"
burned = degrade_pts[degrade_pts["interceding_burn"]]
cond = (burned.t1_rh_98_a0 >= 5) & (burned.t2_rh_98_a0 >= (.65 * burned.t1_rh_98_a0))
with pd.option_context("mode.chained_assignment", None):
    burned["canopy_lost"] = True
    burned.loc[cond, "canopy_lost"] = False
    burned.loc[burned.t1_rh_98_a0 < 5, "canopy_lost"] = np.NaN
print(len(burned[burned.t1_rh_98_a0 < 5]))

fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
axi = ax[0, 0]
axi.scatter(burned.t1_rh_98_a0, burned.t2_rh_98_a0, s=2, c=burned.canopy_lost, cmap="coolwarm")
axi.set_title("RH 98")
maxh = 55
axi.set_xlim(0, maxh)
axi.set_ylim(0, maxh)
axi.plot([0, maxh], [0, maxh], color='gray', linestyle='dashed')
axi.set_xlabel("Time 1")
axi.set_ylabel("Time 2")

axi = ax[0, 1]
axi.scatter(burned[burned.canopy_lost == True].t1_rh_50_a0, burned[burned.canopy_lost == True].t2_rh_50_a0, s=2, color=red)
axi.scatter(burned[burned.canopy_lost == False].t1_rh_50_a0, burned[burned.canopy_lost == False].t2_rh_50_a0, s=2, color=blue)
axi.set_title("RH 50")
maxh = 30
axi.set_xlim(-2.5, maxh)
axi.set_ylim(-2.5, maxh)
axi.plot([0, maxh], [0, maxh], color='gray', linestyle='dashed')
axi.set_xlabel("Time 1")
axi.set_ylabel("Time 2")

sns.set_theme(style="ticks")
axi = ax[1, 0]
sns.kdeplot(x=burned.t1_upper_pai, ax=axi, color='tab:green', fill=True)
sns.kdeplot(x=burned.t2_upper_pai[burned.canopy_lost == False], ax=axi, color=blue1, fill=True)
sns.kdeplot(x=burned.t2_upper_pai[burned.canopy_lost == True], ax=axi, color=red1, fill=True)
axi.set_xlabel("PAI")
axi.set_title("Upper & mid stories")

axi = ax[1, 1]
sns.kdeplot(x=burned.t1_lower_pai, ax=axi, color='tab:green', fill=True)
sns.kdeplot(x=burned.t2_lower_pai[burned.canopy_lost == False], ax=axi, color=blue1, fill=True)
sns.kdeplot(x=burned.t2_lower_pai[burned.canopy_lost == True], ax=axi, color=red1, fill=True)
axi.set_xlabel("PAI")
axi.set_title("Understory")

fig.subplots_adjust(wspace=0.30, hspace=0.5)

legend1_elements = [
        Line2D([0], [0], marker='o', color=blue, label="Canopy preserved",
            markerfacecolor=blue, markersize=7, linestyle='None'),
        Line2D([0], [0], marker='o', color=red, label="Canopy lost",
            markerfacecolor=red, markersize=7, linestyle='None'),
]
fig.legend(handles=legend1_elements, loc='upper center', bbox_to_anchor=(0.5, 0.52),fancybox=False, shadow=False, ncol=2)

legend2_elements = [
        Line2D([0], [0], marker='o', color='tab:green', label="Pre-disturbance",
            markerfacecolor='tab:green', markersize=7, linestyle='None'),
        Line2D([0], [0], marker='o', color=blue1, label="Post-disturbance,\n canopy preserved",
            markerfacecolor=blue1, markersize=7, linestyle='None'),
        Line2D([0], [0], marker='o', color=red1, label="Post-disturbance,\n canopy lost",
            markerfacecolor=red1, markersize=7, linestyle='None'),
    ]
fig.legend(handles=legend2_elements, loc='upper center', bbox_to_anchor=(0.5, 0.07),fancybox=False, shadow=False, ncol=3)
# fig.suptitle("Structural impacts of burning")


In [ ]:

# Percent of [burned forest] in which the canopy was preserved
tot = len(burned[burned.t1_rh_98_a0 >= 5])
sub = len(burned[cond])
print(f"Percent of burned forest overall, in which the canopy was preserved: {sub/tot * 100:.1f}%")

# Percent of [burned samples when the canopy was preserved] in which the understory was lost
tot = len(burned[cond])
sub = len(burned[cond & (burned.t2_rh_50_a0 <= 3.5)])
print(f"Percent of blue points in which the understory was lost: {sub/tot * 100:.1f}%")

# Percent of [burned forest] in which the canopy was preserved but the understory was lost
tot = len(burned[burned.t1_rh_98_a0 >= 5])
sub = len(burned[cond & (burned.t2_rh_50_a0 <= 3.5)])
print(f"Percent of burned forest overall, in which the canopy was preserved but the understory lost: {sub/tot * 100:.1f}%")